# 单细胞数据分析与细胞标注测试

## 测试用例说明

本 notebook 测试一个完整的单细胞数据分析流程：

1. **Supervisor** 选择 `code_dev` agent
2. **Code Dev Agent** 执行单细胞数据分析：
   - 生成 Scanpy 代码
   - 运行得到 UMAP 可视化
   - 进行 Leiden 聚类
   - 在每个聚类上做差异表达分析（DE）
   - 返回每个聚类的 top DE 基因
3. **Supervisor** 选择 `tool_caller` agent
4. **Tool Caller Agent** 使用细胞类型注释工具，根据 DE 基因确定细胞类型
5. **Supervisor** 再次选择 `code_dev` agent
6. **Code Dev Agent** 使用确定的细胞类型进行标注

## 预期结果

- UMAP 可视化图
- Leiden 聚类结果
- 每个聚类的细胞类型标注
- 差异表达基因列表


## 1. 导入必要的库


In [1]:
import os
import sys
from pathlib import Path
import json
from typing import Dict, Any

# 添加项目根目录到路径
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# 导入主图
from src.main import graph
from langchain_core.messages import HumanMessage
import os

# proxy_vars = ["HTTP_PROXY", "HTTPS_PROXY", "ALL_PROXY", "http_proxy", "https_proxy", "all_proxy"]

# for var in proxy_vars:
#     if var in os.environ:
#         del os.environ[var]
#         print(f"已清除环境变量: {var}")


/home/luting/miniconda3/envs/langgraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 准备初始状态


In [2]:
# 定义单细胞数据分析任务
user_query = """
请对单细胞数据进行以下分析：
1. 读取数据并进行预处理（QC、标准化）
2. 进行 PCA 降维
3. 计算 UMAP 可视化
4. 使用 Leiden 算法进行聚类
5. 对每个聚类进行差异表达分析，找出每个聚类的 top 10 差异表达基因
6. 返回每个聚类的差异表达基因列表

数据路径：/home/luting/projects/mas/mas_2/data/pbmc3k.h5ad
输出路径：/home/luting/projects/mas/mas_2/notebooks/result/
"""

# 初始化状态
initial_state = {
    "messages": [HumanMessage(content=user_query)],
    "user_query": user_query.strip(),
    "next_worker": "rag_researcher",  # 初始值，会被 Supervisor 覆盖
    "last_worker": "",
    "final_report": "",
    "code_solution": "",
    "rag_context": "",
    "pending_contribution": None,
    "critique_feedback": None,
    "is_approved": False,
}

print("✅ 初始状态准备完成")
print(f"用户查询: {user_query[:100]}...")


✅ 初始状态准备完成
用户查询: 
请对单细胞数据进行以下分析：
1. 读取数据并进行预处理（QC、标准化）
2. 进行 PCA 降维
3. 计算 UMAP 可视化
4. 使用 Leiden 算法进行聚类
5. 对每个聚类进行差异表达...


## 3. 运行主图并跟踪执行流程


In [ ]:
# 使用 stream 来跟踪执行流程
execution_steps = []
max_steps = 30  # 设置最大步数，防止无限循环

print("🚀 开始执行主图...")
print("=" * 60)

try:
    for step in graph.stream(initial_state, config={"recursion_limit": max_steps}):
        for node_name, node_state in step.items():
            execution_steps.append({
                "node": node_name,
                "state": node_state.copy()
            })
            
            print(f"\n📌 [{len(execution_steps)}] 执行节点: {node_name}")
            
            # 显示关键状态信息
            if "next_worker" in node_state:
                print(f"   → next_worker: {node_state['next_worker']}")
            if "last_worker" in node_state:
                print(f"   → last_worker: {node_state['last_worker']}")
            if "is_approved" in node_state:
                print(f"   → is_approved: {node_state['is_approved']}")
            if "pending_contribution" in node_state and node_state.get("pending_contribution"):
                pending = node_state["pending_contribution"]
                if isinstance(pending, dict) and "code" in pending:
                    code_preview = pending["code"][:200] if len(pending["code"]) > 200 else pending["code"]
                    print(f"   → 生成的代码预览: {code_preview}...")
            
            # 如果到达 finalize，停止
            if node_name == "finalize":
                print("\n✅ 到达 Finalize 节点，任务完成")
                break
    
    print("\n" + "=" * 60)
    print(f"✅ 执行完成，共执行 {len(execution_steps)} 步")
    
except Exception as e:
    print(f"\n❌ 执行出错: {e}")
    import traceback
    traceback.print_exc()


trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b50-7911-ab93-cd8c4cca5b99; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b59-75c2-9999-1797355b78e2; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b59-75c2-9999-17ae2b081948; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b5a-7cb3-9a84-c3d159578a5d; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b5b-78f3-acf2-deea4b7b135e; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b5c-7522-a8a0-0ada0b37a868


🚀 开始执行主图...
--- [Supervisor] 正在做调度决策 ---


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05c-7ef1-9ec7-70fb8e369d4b; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05d-79e0-862e-2faa3793a45f; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05e-7393-a5c7-bce1820c97a9; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05e-7393-a5c7-bcf04c800535; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05f-7982-a6a0-f522a6ff2a02; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a060-7fd2-86db-fd2da144a535; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b59-75c2-9999-1797355b78e2; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b59-75c2-9999-17ae2b081948; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b5a-7cb3-9a84-c3d159578a5d; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-7b5b-78f3-acf2-deea4b7b135e; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id

  --> 决策: code_dev
  --> 理由: 当前任务是完成单细胞数据的完整分析流程（QC、标准化、PCA、UMAP、Leiden聚类、差异表达分析及top10基因提取），所有步骤均需通过代码实现；数据路径和输出路径已明确提供，无需额外文献支持（RAG上下文非必需——单细胞分析流程在Scanpy等标准工具中高度规范化，无模糊概念或方法学争议）；尚未生成任何代码，也无待审核内容；tool_caller适用于调用已有黑盒工具（如细胞类型注释API），但本任务核心是端到端分析流水线开发与执行；因此，应优先由 code_dev 生成并执行符合最佳实践的、可复现的 Python/Scanpy 脚本。

📌 [1] 执行节点: supervisor
   → next_worker: code_dev
   → last_worker: 
   → is_approved: False
--- [Code Dev] 正在生成代码 (迭代 1) ---
  --> 从用户查询中解析到数据路径: /home/luting/projects/mas/mas_2/data/pbmc3k.h5ad
  --> 从用户查询中解析到结果路径: /home/luting/projects/mas/mas_2/notebooks/result/


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.agents.code_dev.executor:数据路径是文件，将挂载目录: /home/luting/projects/mas/mas_2/data, 文件名: pbmc3k.h5ad
INFO:src.agents.code_dev.executor:构建Docker镜像: code-executor-dbb2d861
trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-ee81-7712-b923-d0a45ebc962a; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-ee82-70b3-9a32-0cdef7433b8b; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a05f-7982-a6a0-f522a6ff2a02; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c0939-a060-7fd2-86db-fd2da144a535


...

获取到了 code，前面部分内容：
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import issparse

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi_save=300, facecolor='white')

# Read data
adata = sc.read_h5ad('/app/data/pbmc3k.h5ad', cach
获取到了 requirements.txt，内容：
anndata>=0.8.0
scanpy>=1.9.0
numpy>=1.21.0
pandas>=1.3.0
matplotlib>=3.4.0
seaborn>=0.11.0
scipy>=1.7.0
  --> 代码生成成功，代码长度: 2431 字符
--- [Code Dev] 进行自我检查 ---
--- [Code Dev] 正在执行代码 ---


INFO:src.agents.code_dev.executor:运行容器，镜像: code-executor-bca53d03
INFO:src.agents.code_dev.executor:数据目录挂载: /home/luting/projects/mas/mas_2/data -> /app/data
INFO:src.agents.code_dev.executor:数据文件名: pbmc3k.h5ad (在容器内路径: /app/data/pbmc3k.h5ad)


Using user: 1001:1001


trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c093d-a668-70f3-ba89-9a59980e7347; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c093d-a669-7773-8285-c0d09dc915f6; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c093d-a66a-7e83-bfd4-d8e9358432ce; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c093d-a66a-7e83-bfd4-d8f821eed523; trace=019c0939-7b50-7911-ab93-cd8c4cca5b99,id=019c093c-05a3-75a3-a660-3868d6f62872


【Docker代码执行日志】: /app/code.py:22: FutureWarning: Use `scanpy.set_figure_params` instead
  sc.settings.set_figure_params(dpi_save=300, facecolor='white')
Traceback (most recent call last):
  File "/app/code.py", line 25, in <module>
    adata = sc.read_h5ad('/app/data/pbmc3k.h5ad', cache=False)
TypeError: read_h5ad() got an unexpected keyword argument 'cache'...
  --> 代码执行失败: 未知错误
  --> 执行失败，准备重试 (第 3/3 次)
  --> 设置反馈信息: 代码执行完成，但未找到结果标记\n错误日志摘要：/app/code.py:22: FutureWarning: Use `scanpy.set_figure_params` instead
  sc....
--- [Code Dev] 正在生成代码 (迭代 3) ---


## 4. 提取最终结果


In [ ]:
# 获取最终状态
if execution_steps:
    final_state = execution_steps[-1]["state"]
    
    print("📊 最终状态摘要:")
    print("=" * 60)
    
    # 显示最终答案
    if "final_answer" in final_state:
        print(f"\n📝 最终答案:\n{final_state['final_answer']}")
    
    # 显示代码解决方案
    if final_state.get("code_solution"):
        print(f"\n💻 代码解决方案:\n{final_state['code_solution'][:500]}...")
    
    # 显示 RAG 上下文
    if final_state.get("rag_context"):
        print(f"\n📚 RAG 上下文:\n{final_state['rag_context'][:300]}...")
    
    # 显示最终报告
    if final_state.get("final_report"):
        print(f"\n📄 最终报告:\n{final_state['final_report'][:500]}...")
    
    print("\n" + "=" * 60)
else:
    print("⚠️ 没有执行步骤记录")


📊 最终状态摘要:

💻 代码解决方案:
{'code': 'import os\nimport scanpy as sc\nimport pandas as pd\nimport numpy as np\n\n# 设置结果保存路径\nresult_dir = "./result"\nos.makedirs(result_dir, exist_ok=True)\n\n# 1. 读取数据并进行预处理（QC、标准化）\nprint("Step 1: Loading and preprocessing data...")\nadata = sc.read_h5ad("/home/luting/projects/mas/mas_2/data/pbmc3k.h5ad")\n\n# QC：计算线粒体基因比例（假设mt基因以\'MT-\'或\'mt-\'开头）\nadata.var["mt"] = adata.var_names.str.startswith("MT-") | adata.var_names.str.startswith("mt-")\nsc.pp.calculate_qc_metrics(adata, qc_vars=["...



## 5. 分析执行流程


In [ ]:
# 分析执行流程
print("🔄 执行流程分析:")
print("=" * 60)

node_sequence = [step["node"] for step in execution_steps]
print(f"\n节点执行顺序: {' -> '.join(node_sequence)}")

# 统计各节点执行次数
from collections import Counter
node_counts = Counter(node_sequence)
print(f"\n各节点执行次数:")
for node, count in node_counts.items():
    print(f"  - {node}: {count} 次")

# 查找关键步骤
print("\n🔍 关键步骤:")
for i, step in enumerate(execution_steps):
    node = step["node"]
    state = step["state"]
    
    # Code Dev 生成代码
    if node == "code_dev" and state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            print(f"\n  [{i+1}] Code Dev 生成代码 (步骤 {i+1})")
            print(f"      代码长度: {len(pending['code'])} 字符")
    
    # Tool Caller 调用工具
    if node == "tool_caller" and state.get("tool_name"):
        print(f"\n  [{i+1}] Tool Caller 调用工具: {state['tool_name']} (步骤 {i+1})")
        if state.get("tool_result"):
            result_preview = str(state["tool_result"])[:200]
            print(f"      结果预览: {result_preview}...")
    
    # Critic 审核结果
    if node == "critic":
        is_approved = state.get("is_approved", False)
        status = "✅ 通过" if is_approved else "❌ 驳回"
        print(f"\n  [{i+1}] Critic 审核: {status} (步骤 {i+1})")
        if state.get("critique_feedback"):
            feedback = state["critique_feedback"][:200]
            print(f"      反馈: {feedback}...")


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Missing dependencies for SOCKS support.
trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c043a-2635-79c2-95de-28f15748ea28; trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c0439-e796-7b42-86a6-4c962b9fd0ee; trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c0439-e793-7082-a3dd-696d3b3e719d; trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c0439-e794-7593-8cbd-a75ac7fa7ac6; trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c0439-e795-7e43-844a-34f6cd05f944; trace=019c0435-174d-7ae3-8973-608a8b322ce6,id=019c0435-174d-7ae3-8973-608a8b322ce6


🔄 执行流程分析:

节点执行顺序: supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic -> supervisor -> code_dev -> critic

各节点执行次数:
  - supervisor: 10 次
  - code_dev: 10 次
  - critic: 10 次

🔍 关键步骤:

  [2] Code Dev 生成代码 (步骤 2)
      代码长度: 2582 字符

  [3] Critic 审核: ✅ 通过 (步骤 3)

  [5] Code Dev 生成代码 (步骤 5)
      代码长度: 2922 字符

  [6] Critic 审核: ✅ 通过 (步骤 6)

  [8] Code Dev 生成代码 (步骤 8)
      代码长度: 3592 字符

  [9] Critic 审核: ✅ 通过 (步骤 9)

  [11] Code Dev 生成代码 (步骤 11)
      代码长度: 3688 字符

  [12] Critic 审核: ✅ 通过 (步骤 12)

  [14] Code Dev 生成代码 (步骤 14)
      代码长度: 3950 字符

  [15] Critic 审核: ✅ 通过 (步骤 15)

  [17] Code Dev 生成代码 (步骤 17)
      代码长度: 3058 字符

  [18] Critic 审核: ✅ 通过 (步骤 18)

  [20] Code Dev 生成代码 (步骤 20)
      代码长度: 2866 字符

  [21] Critic 审核:

## 6. 提取代码和结果


In [ ]:
# 提取所有生成的代码
generated_codes = []

for step in execution_steps:
    state = step["state"]
    
    # 从 pending_contribution 提取代码
    if state.get("pending_contribution"):
        pending = state["pending_contribution"]
        if isinstance(pending, dict) and "code" in pending:
            generated_codes.append({
                "step": step["node"],
                "code": pending["code"]
            })
    
    # 从 code_solution 提取代码
    if state.get("code_solution"):
        generated_codes.append({
            "step": "code_solution",
            "code": state["code_solution"]
        })

print(f"📝 找到 {len(generated_codes)} 段生成的代码\n")

for i, code_info in enumerate(generated_codes, 1):
    print(f"代码段 {i} (来源: {code_info['step']}):")
    print("-" * 60)
    print(code_info["code"][:500])
    if len(code_info["code"]) > 500:
        print("...")
    print()


📝 找到 38 段生成的代码

代码段 1 (来源: code_dev):
------------------------------------------------------------
import os
import scanpy as sc
import pandas as pd
import numpy as np

# 设置结果保存路径
result_dir = "./result"
os.makedirs(result_dir, exist_ok=True)

# 1. 读取数据并进行预处理（QC、标准化）
print("Step 1: Loading and preprocessing data...")
adata = sc.read_h5ad("/home/luting/projects/mas/mas_2/data/pbmc3k.h5ad")

# 基础质控：计算线粒体基因比例（假设mt基因以'MT-'或'mt-'开头）
adata.var["mt"] = adata.var_names.str.startswith("MT-") | adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log
...

代码段 2 (来源: code_solution):
------------------------------------------------------------
{'code': 'import os\nimport scanpy as sc\nimport pandas as pd\nimport numpy as np\n\n# 设置结果保存路径\nresult_dir = "./result"\nos.makedirs(result_dir, exist_ok=True)\n\n# 1. 读取数据并进行预处理（QC、标准化）\nprint("Step 1: Loading and preprocessing data...")\nadata = sc.read_h5ad("/home/luting/projects/mas/mas_2/data/pbmc3k.

## 7. 提取细胞类型标注结果


In [ ]:
# 查找 Tool Caller 的细胞类型注释结果
celltype_results = []

for step in execution_steps:
    state = step["state"]
    
    # 查找 tool_caller 的结果
    if step["node"] == "tool_caller":
        tool_result = state.get("tool_result")
        if tool_result and isinstance(tool_result, dict):
            # 检查是否是细胞类型注释结果
            if "final_prediction" in tool_result:
                celltype_results.append({
                    "step": len(celltype_results) + 1,
                    "prediction": tool_result.get("final_prediction"),
                    "decision_logic": tool_result.get("decision_logic"),
                    "details": tool_result.get("details", {})
                })

if celltype_results:
    print("🔬 细胞类型注释结果:")
    print("=" * 60)
    
    for result in celltype_results:
        print(f"\n结果 {result['step']}:")
        print(f"  预测细胞类型: {result['prediction']}")
        print(f"  决策逻辑: {result['decision_logic']}")
        
        if result.get("details"):
            details = result["details"]
            if "enrichr" in details:
                enrichr = details["enrichr"]
                print(f"  Enrichr 结果: {enrichr.get('label', 'N/A')} (置信度: {enrichr.get('confidence', 0):.2f})")
            if "llm_expert" in details:
                llm = details["llm_expert"]
                print(f"  LLM 专家结果: {llm.get('label', 'N/A')} (置信度: {llm.get('confidence', 0):.2f})")
else:
    print("⚠️ 未找到细胞类型注释结果")


⚠️ 未找到细胞类型注释结果


## 8. 保存结果到文件


In [ ]:
# 保存执行结果
output_dir = Path("../results")
output_dir.mkdir(exist_ok=True)

# 保存执行步骤摘要
summary = {
    "total_steps": len(execution_steps),
    "node_sequence": [step["node"] for step in execution_steps],
    "final_state": {
        "final_answer": final_state.get("final_answer", "") if execution_steps else "",
        "code_solution": final_state.get("code_solution", "")[:1000] if execution_steps else "",  # 限制长度
        "rag_context": final_state.get("rag_context", "")[:500] if execution_steps else "",
        "final_report": final_state.get("final_report", "")[:500] if execution_steps else ""
    },
    "generated_codes_count": len(generated_codes),
    "celltype_results_count": len(celltype_results)
}

output_file = output_dir / "single_cell_analysis_result.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print(f"✅ 结果已保存到: {output_file}")

# 保存生成的代码
if generated_codes:
    code_file = output_dir / "generated_code.py"
    with open(code_file, "w", encoding="utf-8") as f:
        for i, code_info in enumerate(generated_codes, 1):
            f.write(f"# Code segment {i} (from {code_info['step']})\n")
            f.write("# " + "="*60 + "\n")
            f.write(code_info["code"])
            f.write("\n\n")
    print(f"✅ 代码已保存到: {code_file}")


✅ 结果已保存到: ../results/single_cell_analysis_result.json
✅ 代码已保存到: ../results/generated_code.py
